In [ ]:
!pip install opendatasets timm lightning --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.1/778.1 kB 46.5 MB/s eta 0:00:00


In [ ]:
import opendatasets as od

# Assign the Kaggle data set URL into variable
dataset = 'https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia/data'
od.download(dataset)

100%|██████████| 2.29G/2.29G [01:46<00:00, 23.2MB/s]


In [ ]:
# custom modules
%load_ext autoreload
%autoreload 2
import utilities
#import model_functions
import model_factory

#lightning modules and callbacks
import lightning_data
import lightning_model
import train_info
import learning_curves
import confusion_matrix

import os

# timm models
import timm

# torch modules (temporarily)
import torch.nn as nn
import torch
# pytorch lightning (for checkpointing callbacks)
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# necessary, as checkpoints will be saved on GDrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ResNet50
'''
arch_name = 'resnet50'
classifier_name = 'linear'
classifier_type = model_factory.get_linear_classifer if classifier_name == 'linear' else model_factory.get_simple_non_linear_classifier
layers_version = 'all'
data_aug_name = "strong"
layers_version_name = layers_version + data_aug_name

model1, config1 = model_factory.get_model(arch_name, classifier_type, layers_version)

check_point_path_1 = '/content/drive/MyDrive/models/resnet50-best/epoch=4-step=370.ckpt'
'''

resnet50_config = {
    'model_name': 'resnet50', # name of the pretrained model
    'classifier_name': 'linear', # name of the classifier (e.g. linear/nonlinear)
    'classifier_type': None, # leave it None
    'layers': 'all', # layers to train (e.g. first (starting from last), second (starting from last), all)
    'augmentation': 'strong', # augmentation type (e.g. normal or strong)
    'classes_weight': None, # weights for each class
    'batch_size': 64,
    'val_split': 0.1,
    'n_epochs': 20,
    'optimizer': 'Adam',
    'scheduler': '', # leave it empty to not use any scheduling
    'ensemble': True,
    'image_size': None,
    'mean': None,
    'std': None
    }

resnet50_config['classifier_type'] = model_factory.get_linear_classifer if config['classifier_name'] == 'linear' else model_factory.get_simple_non_linear_classifier

resnet50_ckpt = '/content/drive/MyDrive/models/resnet50/.../epoch={}-step={}.ckpt'
l_resnet50_model = lightning_model.PneumoniaModel.load_from_checkpoint(resnet50_ckpt, h=resnet50_config)

23,512,130 total parameters.
23,512,130 trainable parameters.


In [ ]:
# DenseNet121
'''
arch_name = 'densenet121'
classifier_name = 'linear'
classifier_type = model_factory.get_linear_classifer if classifier_name == 'linear' else model_factory.get_simple_non_linear_classifier
layers_version = 'all'
data_aug_name = "strong"

model2, config2 = model_factory.get_model(arch_name, classifier_type, layers_version)

check_point_path_2 = '/content/drive/MyDrive/models/densenet121-best/epoch=6-step=518.ckpt'
'''

densenet121_config = {
    'model_name': 'densenet121', # name of the pretrained model
    'classifier_name': 'linear', # name of the classifier (e.g. linear/nonlinear)
    'classifier_type': None, # leave it None
    'layers': 'all', # layers to train (e.g. first (starting from last), second (starting from last), all)
    'augmentation': 'strong', # augmentation type (e.g. normal or strong)
    'classes_weight': None, # weights for each class
    'batch_size': 64,
    'val_split': 0.1,
    'n_epochs': 20,
    'optimizer': 'SGD',
    'scheduler': 'CosineAnnealingLR10', # leave it empty to not use any scheduling
    'ensemble': True,
    'image_size': None,
    'mean': None,
    'std': None
    }

densenet121_config['classifier_type'] = model_factory.get_linear_classifer if config['classifier_name'] == 'linear' else model_factory.get_simple_non_linear_classifier

densenet121_ckpt = '/content/drive/MyDrive/models/densenet121/.../epoch={}-step={}.ckpt'
l_densenet121_model = lightning_model.PneumoniaModel.load_from_checkpoint(densenet121_ckpt, h=densenet121_config)

6,955,906 total parameters.
6,955,906 trainable parameters.


In [ ]:
# EfficientNet_b0
'''
arch_name = 'efficientnet_b0'
classifier_name = 'linear'
classifier_type = model_factory.get_linear_classifer if classifier_name == 'linear' else model_factory.get_simple_non_linear_classifier
layers_version = 'all'
data_aug_name = "strong"

model3, config3 = model_factory.get_model(arch_name, classifier_type, layers_version)

check_point_path_3 = '/content/drive/MyDrive/models/efficientnet-best/epoch=8-step=666.ckpt'
'''

efficientnet_b0_config = {
    'model_name': 'efficientnet_b0', # name of the pretrained model
    'classifier_name': 'linear', # name of the classifier (e.g. linear/nonlinear)
    'classifier_type': None, # leave it None
    'layers': 'all', # layers to train (e.g. first (starting from last), second (starting from last), all)
    'augmentation': 'strong', # augmentation type (e.g. normal or strong)
    'classes_weight': None, # weights for each class
    'batch_size': 64,
    'val_split': 0.1,
    'n_epochs': 20,
    'optimizer': 'SGD',
    'scheduler': '', # leave it empty to not use any scheduling
    'ensemble': True,
    'image_size': None,
    'mean': None,
    'std': None
    }

efficientnet_b0['classifier_type'] = model_factory.get_linear_classifer if config['classifier_name'] == 'linear' else model_factory.get_simple_non_linear_classifier

efficientnet_b0_ckpt = '/content/drive/MyDrive/models/efficientnet_b0/.../epoch={}-step={}.ckpt'
l_efficientnet_b0_model = lightning_model.PneumoniaModel.load_from_checkpoint(efficientnet_b0_ckpt, h=efficientnet_b0)


4,010,110 total parameters.
4,010,110 trainable parameters.


In [ ]:
# create the model
'''
l_model1 = lightning_model.PneumoniaModel.load_from_checkpoint(check_point_path_1, h=config1)
l_model2 = lightning_model.PneumoniaModel.load_from_checkpoint(check_point_path_2, h=config2)
l_model3 = lightning_model.PneumoniaModel.load_from_checkpoint(check_point_path_3, h=config3)

# print(l_model1.test_acc * 100)
# print(l_model2.test_acc * 100)
# print(l_model3.test_acc * 100)
'''

In [ ]:
'''
model1 = l_model1.model
model2 = l_model2.model
model3 = l_model3.model

print(model1)
'''

resnet50_model = l_resnet50_model.model
densenet121_model = l_densenet121.model
efficientnet_b0_model = l_efficientnet_b0_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [ ]:
'''
model1.fc = nn.Sequential(*list(model1.fc.children())[:-1])
model2.classifier = nn.Sequential(*list(model2.classifier.children())[:-1])
model3.classifier = nn.Sequential(*list(model3.classifier.children())[:-1])
'''

resnet50_model.fc = nn.Sequential(*list(resnet50_model.fc.children())[:-1])
densenet121_model.classifier = nn.Sequential(*list(densenet121_model.classifier.children())[:-1])
efficientnet_b0_model.classifier = nn.Sequential(*list(efficientnet_b0_model.classifier.children())[:-1])

In [ ]:
!pip install albumentations --quiet

In [ ]:
'''
config = {
    'model_name': 'efficientnet_b0', # name of the pretrained model
    'classifier_name': 'linear', # name of the classifier (e.g. linear/nonlinear)
    'classifier_type': None, # leave it None
    'layers': 'classifier', # layers to train (e.g. first (starting from last), second (starting from last), all)
    'augmentation': 'normal', # augmentation type (e.g. normal or strong)
    'classes_weight': None, # weights for each class
    'batch_size': 128,
    'val_split': 0.1,
    'n_epochs': 20,
    'optimizer': 'Adam',
    'scheduler': '', # leave it empty to not use any scheduling
    'image_size': None,
    'mean': None,
    'std': None
    }
'''

In [ ]:
# dataloader
pneumonia_data_resnet50 = lightning_data.PneumoniaDataModule(resnet50_config)
pneumonia_data_resnet50.setup()
test_set = pneumonia_data_resnet50.test_set
print(len(test_set))

In [ ]:
# Can I do this?
'''
pneumonia_data.setup()
test_set = pneumonia_data.test_set
print(len(test_set))
'''


624


In [ ]:
import torch
import numpy as np

def create_meta_data(test_loader, model_list):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    meta_X = []
    meta_Y = []

    for inputs, labels in test_loader:
        inputs = inputs.unsqueeze(0)
        inputs = inputs.to(device)

        logits_list = []

        for model in model_list:
            model.to(device)
            model.eval()

            with torch.no_grad():
                logits = model(inputs)

            logits_list.append(logits.cpu().numpy())

        meta_X.append(np.concatenate(logits_list, axis=1))
        meta_Y.append(labels)

    meta_X = np.concatenate(meta_X, axis=0)
    meta_Y = np.array(meta_Y)

    indices = np.arange(meta_X.shape[0])
    np.random.shuffle(indices)

    meta_X = meta_X[indices]
    meta_Y = meta_Y[indices]

    return meta_X, meta_Y

In [ ]:
#models = [model1, model2, model3]
models = [resnet50_model, densenet121_model, efficientnet_b0_model]

X, Y = create_meta_data(pneumonia_data.test_set, models)

In [ ]:
print(X.shape)

(624, 6)


In [ ]:
print(X)

[[-4.312985    4.5012407  -3.2164967   2.694486   -1.7252353   2.0061772 ]
 [-5.0372353   5.251523   -4.092097    5.2101717  -2.74962     2.8244402 ]
 [-2.4904418   2.408436   -2.1623557   1.6424263  -1.1018068   0.9155461 ]
 ...
 [-1.8758922   1.858717    0.55103964 -0.46427637  0.10158892  0.13055673]
 [-3.8166263   3.8185513  -3.2125533   2.8137949  -2.1002035   2.4353764 ]
 [ 4.0108204  -4.0597777   4.0528765  -3.8711917   2.7060137  -2.5737817 ]]


In [ ]:
print(Y.shape)

(624,)


In [ ]:
print(Y)

[1 1 0 1 1 1 0 0 1 0 0 1 0 1 1 1 1 0 1 0 1 0 0 0 0 1 1 1 0 1 1 1 0 0 1 0 0
 0 0 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 0 0 1 1 0 0 1 0
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 1
 0 0 0 1 0 0 1 1 0 1 1 0 1 0 1 0 1 0 0 0 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1
 0 1 0 1 0 1 1 0 0 0 1 0 1 1 0 1 1 1 1 0 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1
 1 1 1 0 1 1 1 1 0 0 1 1 0 0 1 0 1 1 1 1 1 0 0 1 1 1 0 1 1 0 1 1 0 1 0 1 0
 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 0 0 1 0 0 0 1 1 0 0 1 1 1
 1 1 1 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1
 0 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 0 0 0 0 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 1
 1 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 1 1 1 0 1 0 1 1 0 1 1 1 0 0 1 0 0 0 1 1 0
 1 1 0 0 0 0 1 1 0 0 0 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1
 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 0 1 1 0 1 1 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0
 0 1 0 0 1 0 1 0 0 1 1 1 1 1 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 0 0 1 1 0 1 1 0 1 1 1 0 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Assuming X and Y are your features and target variable, respectively

# Step 1: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.8)

# Step 2: Initialize the logistic regression model
model = LogisticRegression()

# Step 3: Train the model on the training set
model.fit(X_train, y_train)

# Step 4: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 5: Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on the test set: {accuracy}")

Accuracy on the test set: 0.944


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Assuming X and Y are your features and target variable, respectively

# Step 1: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.8)

# Step 2: Initialize the SVM model
model = SVC()

# Step 3: Train the model on the training set
model.fit(X_train, y_train)

# Step 4: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 5: Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on the test set: {accuracy}")

Accuracy on the test set: 0.944


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Assuming X and Y are your features and target variable, respectively

# Step 1: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.8, random_state=42)

# Step 2: Initialize the MLP classifier with 2 hidden layers
model = MLPClassifier(hidden_layer_sizes=(1000, 500), max_iter=1000)  # You can adjust the hidden layer sizes and other parameters

# Step 3: Train the model on the training set
model.fit(X_train, y_train)

# Step 4: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 5: Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on the test set: {accuracy}")


Accuracy on the test set: 0.916
